In [8]:
import os
import sys
import re

import numpy as np
import pandas as pd

date = '7_19_2021'

refants = ['DA59','DV11','DA49','DV12','PM01','DV07']

# Data Transfer (Online and Offline TelCal Solutions)

In [9]:
genDir = '/Users/colemeyer/Documents/ALMA Internship/Observational Data/'+date+'/'

for uid in os.listdir(genDir):
    if os.path.isdir(genDir+uid) and uid[:3] == "uid":
        uid = uid.replace("://","___").replace("/","_")

        uidDir = genDir+uid+'/'

        numOfObs = 0
        for file in os.listdir(genDir):
            if os.path.isdir(genDir+file) and file[:3] == 'uid':
                numOfObs += 1

        antXML = open(uidDir+'Antenna.xml',"r")
        statXML = open(uidDir+'Station.xml',"r")
        weaXML = open(uidDir+'Weather.xml',"r")
        antArr = pd.DataFrame(columns = ['antname','padname','statnum','poserrX','poserrY','poserrZ','poserr()X','poserr()Y','poserr()Z','newposX','newposY','newposZ','geoerrX','geoerrY','geoerrZ','geoposX','geoposY','geoposZ','delayrms'])
        weaArr = pd.DataFrame(columns = ['padname','statnum','geoposX','geoposY','geoposZ','pressure','relHum','temp','windDir','windSpeed','windMax'])

        ##########
        line = antXML.readline()
        while "<row>" not in line:
            line = antXML.readline()
        ##########
        line = statXML.readline()
        while "<row>" not in line:
            line = statXML.readline()
        ##########
        line = weaXML.readline()
        while "<ContainerEntity" not in line:
            line = weaXML.readline()

        ########## Antenna.xml Cycling

        line = antXML.readline()
        first = True
        while "</AntennaTable>" not in line:
            temp = np.empty(19,dtype='U25')
            if not first: line = antXML.readline() 
            else: first = False
            temp[0] = re.sub(r"<.*?>", "", antXML.readline()).strip()
            for i in range(6): line = antXML.readline()
            temp[2] = re.sub(r"<.*?>", "", antXML.readline()).strip()
            for i in range(2): line = antXML.readline()

            antArr = antArr.append(pd.Series(temp,index=antArr.columns),ignore_index=True)
        
        ########## Station.xml Cycling

        k = 0
        line = statXML.readline()
        first = True
        while "</StationTable>" not in line:
            temp = np.empty(5,dtype='U25')
            if not first: line = statXML.readline()
            else: first = False
            temp[4] = re.sub(r"<.*?>", "", line).strip()
            temp[0] = re.sub(r"<.*?>", "", statXML.readline()).strip()
            line = statXML.readline().split()
            for i in range(1,3): temp[i] = line[i+1]
            temp[3] = line[4].rstrip("</position>\n")
            for i in range(3): line = statXML.readline()

            if "A" in temp[0]:
                antArr.at[k, 'padname'] = temp[0]
                antArr.at[k,['geoposX','geoposY','geoposZ']] = temp[1:4]
            else:
                temp2 = np.empty(11,dtype='U25')
                temp2[0] = temp[0]
                temp2[1] = temp[4]
                temp2[2:5] = temp[1:4]
                weaArr = weaArr.append(pd.Series(temp2,index=weaArr.columns),ignore_index=True)

            k += 1
        
        ########## .Log Processing

        logDir = os.listdir(uidDir)
        for name in logDir:
            if name[-4:] == ".log":
                log = open(uidDir+name,"r")


        line = log.readline()
        while "Start:" not in line:
            line = log.readline()

        logArr = pd.DataFrame(columns = ['poserrX','poserrY','poserrZ','poserr()X','poserr()Y','poserr()Z','newposX','newposY','newposZ','geoerrX','geoerrY','geoerrZ','geoposX','geoposY','geoposZ','delayrms'])

        for i in range(2): line = log.readline()
        while "==" not in line:
            temp = np.empty(16,dtype='U25')
            line = line.split()
            for i in range(3): temp[i] = line[2*(i+2)]
            for i in range(2): temp[i+3] = line[2*(i+3)-1][1:-3]
            temp[5] = line[9][1:-2]
            temp[15] = line[12]
            line = log.readline().split()
            for i in range(3): temp[i+6] = line[2*(i+1)]
            line = log.readline().split()
            for i in range(3): temp[i+9] = line[2*(i+1)]
            line = log.readline().split()
            for i in range(3): temp[i+12] = line[2*(i+1)-1]
            line = log.readline()

            logArr = logArr.append(pd.Series(temp,index=logArr.columns),ignore_index=True)
        
        for i in range(1,antArr.shape[0]):
            antArr.iloc[i,3:16] = logArr.iloc[i-1,:-1]
            antArr.iloc[i,-1] = logArr.iloc[i-1,-1]
        
        pd.DataFrame(antArr).to_csv(uidDir+'AntPos.csv', index=None)

        ########## Weather.xml Processing

        weaArr.index = weaArr.statnum

        statNums = pd.Series(weaArr.statnum).reset_index(drop=True)
        weaData = np.empty((statNums.shape[0],15,6),dtype='U25')

        line = weaXML.readline()
        while "</WeatherTable>" not in line:
            temp = np.empty(6,dtype='U25')
            line = weaXML.readline()
            for i in range(6): temp[i] = re.sub(r"<.*?>", "", weaXML.readline()).strip()
            stat = str(weaXML.readline().lstrip("    <stationId>").rstrip("</stationId>\n"))
            index = statNums[statNums == stat].index.tolist()[0]
            nextIndex = np.count_nonzero(weaData[index,:,0] != "")

            line = weaXML.readline()
            line = weaXML.readline()
            weaData[index,nextIndex,:] = temp[:]

        weaData[weaData == ""]=np.nan
        for i in range(statNums.shape[0]):
            for j in range(6):
                newArr = np.array(weaData[i,:statNums[statNums == stat].index.tolist()[0],j],dtype='float')
                weaArr.iat[i,j+5] = np.average(newArr)

        pd.DataFrame(weaArr).to_csv(uidDir+'WeaData.csv', index=None)

        ########## Data Reduction

        refinedAntArr = pd.DataFrame(columns=antArr.columns)

        for i in range(antArr.shape[0]):
            if antArr.at[antArr.index[i], 'antname'] in refants:
                refinedAntArr = refinedAntArr.append(antArr.iloc[i],ignore_index=True)

        refinedAntArr.index = refinedAntArr['antname']
        refinedAntArr = refinedAntArr.reindex(refants)

        pd.DataFrame(refinedAntArr).to_csv(uidDir+'AntPosRefined.csv', index=None)

# Observation Combination

In [16]:
antNames = pd.Series(refants)
antData = np.empty((antNames.shape[0],numOfObs,21),dtype='U25')
weaNames = pd.Series(['Meteo129','Meteo130','Meteo131','Meteo201','Meteo309','Meteo410','MeteoCentral','MeteoItinerant'])
weaData = np.empty((weaNames.shape[0],numOfObs,10),dtype='U25')

k = 0
for uid in os.listdir(genDir):
    if os.path.isdir(genDir+uid) and uid[:3] == "uid":
        antArr = pd.read_csv(genDir+uid+'/AntPosRefined.csv').to_numpy(dtype='U25')
        weaArr = pd.read_csv(genDir+uid+'/WeaData.csv').to_numpy(dtype='U25')
        
        mainXML = open(genDir+uid+'/Main.xml',"r")
        for i in range(6): line = mainXML.readline()
        time = re.sub(r"<.*?>", "", line).strip()
        
        for i in range(antNames.shape[0]):
            antData[i,k,:2] = np.array([time,uid],dtype='U25')
            antData[i,k,2:] = antArr[i,:]
            
        for i in range(8):
            weaData[weaNames[weaNames == weaArr[i,0]].index.tolist(),k,1:] = weaArr[weaNames[weaNames == weaArr[i,0]].index.tolist(),2:]
            weaData[weaNames[weaNames == weaArr[i,0]].index.tolist(),k,0] = time
            
        k += 1
    
for i in range(8): 
    temp = pd.DataFrame(weaData[i,(weaData[i,:,0].argsort())],columns=['time','geoposX','geoposY','geoposZ','pressure','relHum','temp','windDir','windSpeed','windMax'])
    pd.DataFrame(temp).to_csv(genDir+weaNames[i]+'.csv', index=None)
    
for i in range(6): 
    temp = pd.DataFrame(antData[i,(antData[i,:,0].argsort())],columns=['time','uid','antname','padname','statnum','poserrX','poserrY','poserrZ','poserr()X','poserr()Y','poserr()Z','newposX','newposY','newposZ','geoerrX','geoerrY','geoerrZ','geoposX','geoposY','geoposZ','delayrms'])
    pd.DataFrame(temp).to_csv(genDir+refants[i]+'.csv', index=None)